# COCO Project 2024

## ReadMe

Address the following points in a clear and concise manner:
1. How the failure scenario has been modeled in the simulation. 
2. What parameters are used to demonstrate the failure scenario. 
3. How the proposed model-based controller is implemented. 
4. What parameters need to be tuned for the model-based controller.
5. How you tuned the parameters for the model-based controller and how you would recommend tuning them.
6. How the proposed data-driven controller is implemented. 
7. What parameters need to be tuned for the data-driven controller.
8. How you tuned the parameters for the data-driven controller and how you would recommend tuning them. 
9. If you include an extra slide, a clear explanation of the implementation considerations of your bonus AP controller.

### PID:
#### 1. PID Failure Scenario
- If our patients wakes up very late and decides still have a breakfast, altough lunch is just one hour later, our PID controller will fail. Instead of a TIR of 100% with normal diet, it achieves an TIR accuracy of only 84,6%. As the plot shows, the PID controller also manages to steer the blood glucose value into the Hypogycemia range, which is a life threatning state for our patient. The reason for this is that the PID controller has a big Kp proportional and a big derivative Kd value. After seeing that the BG value was too high, it gave a large control effort to steer it back to the steady-state xss. However, also with the effect of the delay, the basal insulin input was too high. This steers the BG value to the Hypoglycemia range.

#### 2. Parameters of the PID controller
- Kp = 0.0002
- Kd = 0.005
- Ki = 0.0



### MPC:
#### 3. MPC controller implementation
- Q = np.diag([10]*8 + [10])
- r = 1000
- Q_ter = np.diag([10000]*8 + [10000])
- K = 35
- disc_step = 8

#### 4. Parameters of the MPC controller

#### 5. Recommendation for tuning the MPC parameters



### DeePC:
#### 6. DeePC controller implementation
- Q = np.diag([10]*8 + [10]) -> This is the cost weight for the states
- r = 1000
- Q_ter = np.diag([10000]*8 + [10000])
- K = 35
- disc_step = 8

lambda_g = 10000
    lambda_o = 1000
        'r' : 100,  # input cost weight in the DeePC optimization
    'T_fut' : 10,  # number of timesteps into the future that the DeePC optimization plans for. Also, L = T_past + T_fut
    'T_past' : 9,  # number of timesteps into the past that the DeePC optimization plans for. ######## TODO: Changing this from 10 to 20 broke my code!!! 
    'T' : T,       # total number of timesteps used to build the Hankel matrix
    'n' : np.shape(A)[0],  # num of states: 9
    'm' : np.shape(B)[1],  # num of inputs: 2
    'p' : np.shape(C)[0],  # num of outputs: 2
    'which_tau' : 0,
    'input_past' : [],
    'output_past' : [],
    'discretization' : 10,
    'x0_old' : 0
#### 7. Parameters of the DeePC controller

#### 8. Recommendation for tuning the DeePC parameters


In [1]:
################################### Necessary Imports ##########################################
# all the necessary imports
################################################################################################

# Module for generationg the metrics
from py_agata.py_agata import Agata # class Agata:  Attributes: glycemic_target: str, Methods: analyze_glucose_profile() -> Runs ReplayBG
from py_agata.time_in_ranges import time_in_target
from py_agata.utils import glucose_time_vectors_to_dataframe

# Function to plot the results
from plot_results import plot_results

# Function to run the simulation
from replay_bg_wrapper import simulate
from py_replay_bg.model.t1d_model import T1DModel
from py_replay_bg.py_replay_bg import ReplayBG


# Python imports
import pandas as pd
import os
import numpy as np
from random import randint

# Convex optimization problems with Python
from cvxpy.reductions.solvers.defines import QP_SOLVERS
import cvxpy as cp
from random import randint

ModuleNotFoundError: No module named 'py_agata'

## PID Controller

The following code initializes Insulin Co's PID controller. 

In [ ]:
class PIDController:
    def __init__(self, Ki, Kd, Kp):
        self.Ki = Ki
        self.Kd = Kd
        self.Kp = Kp
        self.integral = 0
        self.prev_error = 0
    
    def compute_pid(self, model, x0, tau, time_index, inputs_, xss, uss, dss):
        # Error calculation from the desired setpoint
        error = x0[8]-xss[8]
        
        # Integral, Derivative, and Proportional terms of PID
        self.integral += error * 1
        derivative = (error - self.prev_error) / 1
        self.prev_error = error
        
        # Calculate PID control output
        control_output = self.Ki * self.integral + self.Kd * derivative + self.Kp * error + uss
        
        # Ensure control output is within valid limits (e.g., non-negative)
        control_output = max(0, control_output)

        return control_output, dss

    def restart(self):
        self.integral = 0
        self.prev_error = 0


In [ ]:
######################## additional functions for ReplayBG's T1DModel class  ###################
# Here define some additional functions for the T1DModel class that you may find useful.
################################################################################################

def get_discretized_meals(self, initial_time_step, duration, T):
    """
    Parameters
    ----------
    initial_time_step : initial time step of the intended interval that we want to get the meals from
    duration : duration of the interval that we want to get the meals from
    T : discretization step
    
    Returns
    -------
    meal : a vector with all the meals in the time interval given
    """
    meal = np.zeros((duration//T, ))
    for k in range((duration)):
        meal[k//T] += self.meal_ALL_delayed[initial_time_step+k]
    return meal


def get_linearization(self, T, x):
    """
    Provides the linearization parameters, given the discretization interval and the state at which the linearization is done.
    
    Parameters
    ----------
    T : discretization step !!!!!-> 1min!!!!!
    x : linearization point
    
    Returns
    -------
    A, B, h : discretized linear state equations
    """

    x0 = x[0]
    x1 = x[1]

    mp = self.model_parameters

    k1_T = 1 / (1 + T*mp.kgri)
    k2_T = T*mp.kgri / (1 + T*mp.kempt)
    k3_T = 1 / (1 + T*mp.kempt)
    kb_T = 1 / (1 + T*mp.kabs_avg)
    ki1_T = 1 / (1 + T*mp.kd)
    ki2_T = 1 / (1 + T*mp.ka2)
    kie_T = 1 / (1 + T*mp.ke)

    x1_c_T = mp.p2 * mp.SI / mp.VI * T/(1+mp.p2*T)
    x4_c_T = T*mp.kempt * kb_T
    x6_c_T = T*mp.kd * ki2_T
    x7_c_T = T*mp.ka2 * kie_T

    df_dx0 = -mp.SG - x1 * (1 + mp.r1)
    df_dx1 = -x0 * (1 + mp.r1)
    df_dx4 = (mp.f * mp.kabs_avg) / mp.VG
    dx0_dt = -mp.SG*x0 -x0*x1*(1 + mp.r1) + mp.SG*mp.Gb
    div = (1-T*df_dx0)

    #                      x0               x1            x2      x3         x4        x5           x6            x7         x8
    self.Alinear[:] =  [ [1/div         , T*df_dx1/div , 0   , 0     , T*df_dx4/div, 0     , 0             , 0           , 0             ],
                         [0             , 1/(1+mp.p2*T), 0   , 0     , 0           , 0     , x1_c_T*x7_c_T , x1_c_T*kie_T, 0             ],
                         [0             , 0            , k1_T, 0     , 0           , 0     , 0             , 0           , 0             ],
                         [0             , 0            , k2_T, k3_T  , 0           , 0     , 0             , 0           , 0             ],
                         [0             , 0            , 0   , x4_c_T, kb_T        , 0     , 0             , 0           , 0             ],
                         [0             , 0            , 0   , 0     , 0           , ki1_T , 0             , 0           , 0             ],
                         [0             , 0            , 0   , 0     , 0           , x6_c_T, ki2_T         , 0           , 0             ],
                         [0             , 0            , 0   , 0     , 0           , 0     , x7_c_T        , kie_T       , 0             ], 
                         [T/(T+mp.alpha), 0            , 0   , 0     , 0           , 0     , 0             , 0           , mp.alpha/(T+mp.alpha)]]

    self.Blinear[:] = [0, 0, 0, 0, 0, T/(1+T*mp.kd)*mp.to_mgkg, 0, 0, 0]
    self.Mlinear[:] = [0, 0, k1_T, 0, 0, 0, 0, 0, 0]
    self.hlinear[:] = [ T*(dx0_dt - x0*df_dx0 - x1*df_dx1)/div, 0, 0, 0, 0, 0, 0, 0, 0]

    return self.Alinear, np.vstack((self.Blinear, self.Mlinear)).T, self.hlinear

In [ ]:
##################################### Linearization example  ###################################
# A demonstration of T1DModel and get_linearization()
################################################################################################

# State Definition
x0 = np.array([120.0, 0.0016, 0.0, 0.0, 0.0, 1.5081, 10.2735, 0.2827, 120.0])
tau = 8 # min of delay for controler to act

#Some parameters of the patient (everything is set in the simulation for your controllers. These are only an example)
bw = 62.0 # Weight
is_single_meal = False
data = pd.read_csv(
        os.path.join(os.path.abspath(''), 'data', 'data_cho.csv'))
data.t = pd.to_datetime(data['t'], format='mixed')

t1d_model = T1DModel(data=data, bw=bw, yts=5, glucose_model='IG', is_single_meal=False, exercise=False)

# To design the controller two function will be needed:
# - t1d_model.get_linearization
# - t1d_model.get_discretized_meals 

# Example of running the function for getting the system equations
A_, B_, h_ = t1d_model.get_linearization(1, x0)
#print("A:\n", A_.round(3), "\n"*2, "B:\n", B_.round(3), "\n"*2, "h:\n", h_.round(3))
i = 0.0
m = 0.0

### Task 1: Failure Mode

Demonstrate a compelling failure scenario in which InsulinCo's PID controller fails to successfully regulate insulin to motivate a better controller.

In [ ]:
# If the patient eats the meals too often, or in all at once, the simple PID controller fails to keep the BG level. 
# As one can see in the plot below, the patient's blood glucose levels are both in the Hyperglycemia and in the Hypoglycemia zones, 
# which endangeres the patients health.

##################################### Demonstration of InsulinCo's PID control  ###################################

basal_handler_params = dict()
basal_handler_params['default_rate'] = 0.01

# Get test data
data_given_fail = pd.read_csv(
        os.path.join(os.path.abspath(''), 'data', 'data_cho_pid_failure.csv'))

# Instatiating PID Controller
pid_controller = PIDController(Ki=0.0, Kd=0.005, Kp=0.0002)

# Running the simulation
glucose, i, insulin_bolus, m, time = simulate(basal_handler=pid_controller.compute_pid,
       basal_handler_params=basal_handler_params, data_given=data_given_fail,
       meal_input_modulation_factor=1)

data = glucose_time_vectors_to_dataframe(glucose=glucose, t=time)

agata = Agata()
results = agata.analyze_glucose_profile(data)
print(results)

TimeInTarget = time_in_target(data)
print('TimeInTarget = ' + str(TimeInTarget) + '%')

plot_results(glucose=glucose,insulin_bolus=insulin_bolus, i=i, m=m, time=time)

NameError: name 'pd' is not defined

### Task 2: Model-Based Controller

Propose and implement a Model-Based controller, assuming that you access to the (linearized) system model described above and a full state measurement.

In [ ]:
################################### MPC Variables ################################################ 
# State Definition
x0 = np.array([120.0, 0.0016, 0.0, 0.0, 0.0, 1.5081, 10.2735, 0.2827, 120.0])
tau = 8 

#Some parameters of the patient
bw = 62.0
is_single_meal = False

data_given_mpc = pd.read_csv(
        os.path.join(os.path.abspath(''), 'data', 'data_cho_pid_failure.csv'))
data_given_mpc.t = pd.to_datetime(data_given_mpc['t'], format='mixed')

t1d_model = T1DModel(data=data_given_mpc, bw=bw, yts=5, glucose_model='IG', is_single_meal=False, exercise=False)

i = 0.0
m = 0.0

In [ ]:
def mpc_controller(model, x0, tau, time_index, inputs_, xss, uss, dss): 
    Q = dss.basal_handler_params['Q']
    r = dss.basal_handler_params['r']
    prev_u = dss.basal_handler_params['prev_u']
    K = dss.basal_handler_params['K']
    disc_step = dss.basal_handler_params['disc_step']
    Q_ter = dss.basal_handler_params['Q_ter']

    if  time_index % disc_step == 0: 

        cost = 0
        constr = []
        result = 0
    
        A_delay_disc, B_delay_disc, h_delay_disc = t1d_model.get_linearization(tau, x0)
            
        prev_applied_u = np.sum(inputs_[time_index : time_index + tau])
        prev_applied_meals = model.get_discretized_meals(initial_time_step=time_index, duration=tau, T=tau)[0]
        prev_applied_inputs = np.vstack((prev_applied_u, prev_applied_meals))
        prev_applied_inputs = cp.reshape(prev_applied_inputs, (2))
        x0_plus_tau_steps = A_delay_disc @ x0 + B_delay_disc @ prev_applied_inputs + h_delay_disc
    
        x = cp.Variable((9, K + 1))
        u = cp.Variable(K)
        x8_slack = cp.Variable(K, nonneg=True)
    
        A, B, h = t1d_model.get_linearization(disc_step, xss)
        meals = model.get_discretized_meals(initial_time_step=time_index+tau, duration=K*disc_step, T=disc_step)
    
        for k in range(K):
            cost += cp.quad_form(x[:, k] - xss, Q) + cp.sum_squares(u[k] - uss)*r
            inputs = cp.vstack((u[k], meals[k]))
            inputs = cp.reshape(inputs, (2))
            constr += [x[:,k+1] == A @ x[:,k] + B @ inputs + h]
            constr += [x[8, k] >= 70, x[8, k] <= 180 + x8_slack[k]]
            constr += [u[k] <= 0.04, u[k] >= 0.0]
        
        constr += [x[:,0] == x0_plus_tau_steps]
        cost += cp.quad_form(x[:, K] - xss, Q_ter)
    
        problem = cp.Problem(cp.Minimize(cost), constr)
        problem.solve(solver='CLARABEL')
        
        result = u[0].value
        print("##### ", x[8,0].value, "#####")
        if problem.status != cp.OPTIMAL:
            print("################ Resorting to Backup Controller [!]")
            if x0[8] < 190.0: 
                result = 0.0
            else:
                result = 0.01
            print(f"Status at {time_index} is {problem.status}")
            print(f"problem result at time index {time_index}: result {result} #########################")
            basal_handler_params_mpc['prev_u'] = result

        else: 
            if result < 0:
                result = 0
            print(f"Status at {time_index} is {problem.status}")
            print(f"new result at time index {time_index}: result {result} #########################")
            basal_handler_params_mpc['prev_u'] = result

    else: 
        result = prev_u
        print(f"prev result at time index {time_index}: result {result}")
    
    return result, dss


############################################ Simulate #################################################

# Define MPC controller parameters
basal_handler_params_mpc = {
    'Q': np.diag([10]*8 + [10]),  
    'r': 1000,
    'Q_ter' : np.diag([10000]*8 + [10000]),
    'prev_u' : 0,
    'K' : 35,
    'disc_step' : 8
}

# Running the simulation with MPC controller
glucose, i, insulin_bolus, m, time = simulate(
    basal_handler=mpc_controller,
    basal_handler_params=basal_handler_params_mpc,
    data_given=data_given_mpc,
    meal_input_modulation_factor=1
)

# Process and analyze results
data = glucose_time_vectors_to_dataframe(glucose=glucose, t=time)

agata = Agata()
results = agata.analyze_glucose_profile(data)
print(results)

TimeInTarget = time_in_target(data)
print('TimeInTarget = ' + str(TimeInTarget) + '%')

plot_results(glucose=glucose, insulin_bolus=insulin_bolus, i=i, m=m, time=time)

### Task 3: Data-enabled Predictive Control

Propose and implement a DeePC controller, assuming that you measure the glucose $g(t)$ and the “non-monomatic” or “inactive insulin” $i_{sc1}$ (in practice, $i_{sc1}$ cannot be measured. Removing this measurement is one option for the bonus). Use the data that InsulinCo has provided to you as the training data.

The file `DeePC_data.pkl` contains a pandas data frame with 4 columns:
- $m(t)$ (u[1]): Carbohydrate intake/"meals" (uncontrolled)
- $i(t)$ (u[0]): Exogenous/Basal insulin injections (controlled)
- $i_{sc1}(t)$ (x[5]): the insulin in a non-monomeric state 
-  $g(t)$ (x[8]): the interstitial glucose concentration

Every row is a different time step corresponding to 1 minute intervals.

To discretize the meals in a way that is equivalent to `get_discretized_meals`, you should add all the meals in a given interval. For example, if one uses a time-discretization of 5 minutes, the meals vector entry for the first control timestep should be m[0] = sum(meal[0:5]), and for the second control timestep m[1] = sum(meal[5:10]). For the input, you might want to use the average, i.e., i[0] = avg(input[0:5]), though that is a design decision that is up to you.

Also note that you may not need to use all the timesteps in the file.

To load the dataframe, you can use the following line:
`df = pd.read_pickle("DeePC_data.pkl")`
which loads the dataframe into the variable "df".

To solve the optimization problem, we advise you to use the MOSEK solver ('problem.solve(solver=cp.MOSEK'), though you are welcome to use other solvers.

## DeePC controller with normal cho

In [ ]:
def get_datasheet_data(u_data_whole, y_data_whole, A, B, C, T):
    unpickled_df = pd.read_pickle("DeePC_data.pkl") 
    input = unpickled_df["input"]
    meal = unpickled_df["meal"]
    x_5 = unpickled_df["x_5"]
    x_8 = unpickled_df["x_8"]
    
    u_data_list = []
    y_data_list = []
    
    for i in range(T):
        u_data = np.array([[input[i]], [meal[i]]])
        y_data = np.array([[x_5[i]], [x_8[i]]])
        
        u_data_list.append(u_data)
        y_data_list.append(y_data)

    # Convert the lists to numpy arrays and ensure correct shape
    u_data_whole = np.array(u_data_list)
    y_data_whole = np.array(y_data_list)

    return u_data_whole, y_data_whole

def get_Hankel_matrices(T, L, m, p):
    num_hankel_cols = T - L + 1  # 150 - 20 + 1 = 131
    
    H_u = np.zeros((m * L, num_hankel_cols))
    H_y = np.zeros((p * L, num_hankel_cols))
    
    for i in range(num_hankel_cols):
        H_u[:, i] = u_data[i:i+L, :].flatten()
        H_y[:, i] = y_data[i:i+L, :].flatten()
        
    print("H_u: ", H_u.shape)  # Expected shape: (40, 131)
    print("H_y: ", H_y.shape)  # Expected shape: (40, 131)

    ###### rank of Hankel matrix must be at least: mL+n = 2*20 + 9 = 49 ! ##############
    
    return H_u, H_y

def interleave_cvpy(T_custom, u, meal):
    '''
    This function puts u and meal interleaved in one cvxpy list!
    '''
    input_list = []
    for f in range(T_custom):
        input_list.append(u[f])
        input_list.append(meal[f])
    input = cp.hstack(input_list)
    input = cp.reshape(input, (2 * T_custom))
    return input

In [ ]:
######################################## MIMO DeePC Controller Parameter ########################################
# State Definition
x0 = np.array([120.0, 0.0016, 0.0, 0.0, 0.0, 1.5081, 10.2735, 0.2827, 120.0])

#Some parameters of the patient
bw = 62.0
is_single_meal = False

i = 0.0
m = 0.0

# Get cho
data_given_deepc = pd.read_csv(os.path.join(os.path.abspath(''), 'data', 'data_cho.csv'))
data_given_deepc.t = pd.to_datetime(data_given_deepc['t'], format='mixed')
t1d_model = T1DModel(data=data_given_deepc, bw=62.0, yts=5, glucose_model='IG', is_single_meal=False, exercise=False)

discr_step_deepc = 1
A, B, h = t1d_model.get_linearization(discr_step_deepc, x0) # our system now has the same discretization interval as the measurement CGM delay tau!

# Output y dynamics
C = np.array([[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1]]) # we measure x[5] and x[8]!

# System constants
n = np.shape(A)[0]  # num of states: 9
m = np.shape(B)[1]  # num of inputs: 2
p = np.shape(C)[0]  # num of outputs: 2

r = 0.0001  # input cost weight in the DeePC optimization
T_past = 9  # number of timesteps into the past that the DeePC optimization considers
T_fut = 10  # number of timesteps into the future that the DeePC optimization plans for. This is also the prediction horizon!!!

T_init = n
T = 3*(T_past + T_fut) + n - 1 #150
L = T_past + T_fut 

if T<(m+1)*(T_init+T_fut) + n - 1:
    raise Exception(f'T need to be bigger! T is now: {T} and must be at least {(m+1)*(T_init+T_fut) + n - 1}')
########################################################################################################

# Collect data
u_data = np.zeros((T, 2))
y_data = np.zeros((T, 2))

u_data, y_data = get_datasheet_data(u_data, y_data, A, B, C, T)  # these are np.arrays!

#print("u_data: ", u_data.shape)  # Expected shape: (T, 2)
#print("y_data: ", y_data.shape)  # Expected shape: (T, 2)

# Get Hankel matrices
H_u, H_y = get_Hankel_matrices(T, L, m, p)

In [ ]:
################### FINAL VERSION DEEPC ##############################################
#################################################### DeePC Controller ##################################################
def deepc_controller(model, x0, tau, time_index, inputs_, xss, uss, dss): 
    r = dss.basal_handler_params['r'] 
    T_fut = dss.basal_handler_params['T_fut'] 
    T_past = dss.basal_handler_params['T_past'] 
    T = dss.basal_handler_params['T']  
    n = dss.basal_handler_params['n']  
    m = dss.basal_handler_params['m']  
    p = dss.basal_handler_params['p'] 
    which_tau = dss.basal_handler_params['which_tau']
    input_past = dss.basal_handler_params['input_past']
    output_past = dss.basal_handler_params['output_past']
    discretization = dss.basal_handler_params['discretization']
    x0_old = dss.basal_handler_params['x0_old']
    
    L = T_past + T_fut    

    # Get tau
    if inputs_[time_index] == 0:
        which_tau += 1
        dss.basal_handler_params['which_tau'] = which_tau
        print("which_tau: ", which_tau)
        
    # We don't have T_past values of inputs and outputs yet! Generate vectors of 2*T_past!
    # Not only for the first iteration, but for the first T_past iterations! (if T_past>tau!!)
    if time_index <= T_past:
        input_past = np.tile(np.array([[uss], [0]]), (T_past, 1)).reshape(2 * T_past)
        output_past = np.tile(np.array([[xss[5]], [xss[8]]]), (T_past, 1)).reshape(2 * T_past)
                
        dss.basal_handler_params['input_past'] = input_past
        dss.basal_handler_params['output_past'] = output_past
    else:
        # Since we have the initial values of input_past and output_past, we now want real values!
        # Every new iteration, we update a new value for our vectors!
        meal_past = model.get_discretized_meals(initial_time_step=time_index-1, duration=1, T=1)
        input_past = np.roll(input_past, -m)
        ##### TODO: tau or no tau here below?????
        input_past[-m] = inputs_[time_index - 1] # not -T_past! T_past is just the length of input_past! But the value is just the previous one!
        input_past[-m+1] = meal_past[0]
        
        output_past = np.roll(output_past, -p)
        output_past[-p] = x0_old[5]
        output_past[-p+1] = x0_old[8]
        
        dss.basal_handler_params['input_past'] = input_past
        dss.basal_handler_params['output_past'] = output_past
    
    # Cvxpy Variables
    u = cp.Variable(m*T_fut)
    y = cp.Variable(p*T_fut)
    g = cp.Variable(T-L+1) 
    slack_hyper = cp.Variable((T_fut), nonneg=True)
    sigma = cp.Variable((2*T_past))

    lambda_g = 10000
    lambda_o = 1000
    
    cost = 0
    constraints = []
    
    y_ref = np.array([xss[5], xss[8]])

    for j in range(T_fut):
        cost += 10*cp.quad_form(y[p*j:p*j+p]-y_ref, np.eye(2)) + r*cp.quad_form(u[m*j:m*j+m]-uss,np.eye(2))
        cost += lambda_g * cp.norm(g, 1) 
        cost += lambda_o * cp.norm(sigma, 1)
        
    # Partioning
    U_p = H_u[:m*T_past]
    U_f = H_u[-m*T_fut:]
    Y_p = H_y[:p*T_past]
    Y_f = H_y[-p*T_fut:]
    
    # debug
    test = Y_p @ g
    #print("shapes of Y_p, g, y_past: ", Y_p.shape, g.shape, output_past.shape, test.shape)

    constraints += [
        U_p @ g == input_past,
        U_f @ g == u,
        Y_p @ g == output_past + sigma,
        Y_f @ g == y
    ]

    meal_fut = model.get_discretized_meals(initial_time_step=time_index, duration=T_fut, T=1)
    
    for h in range(T_fut):
        if h < tau:
            constraints += [u[2*h] == inputs_[time_index + h]] # Delay constraints
        constraints += [u[2*h] >= 0.0, u[2*h] <= 0.04]
        constraints += [y[2*h+1] >= 70.0] # this is the blood glucose value!
        constraints += [u[2*h+1] == meal_fut[h]]
        constraints += [y[2*h+1] <= 180 + slack_hyper[h]] # state constraints soft upper boundary

    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve(solver='CLARABEL', verbose=False)
    #constraints += [y[1]>=70.0]
    print("blood glucose y[1] value : ###########", y[1].value, "################")
    
    u_fut = u[2*which_tau].value ########### TODO: Should we really use tau??? ###############

    if prob.status != 'optimal':
        print(f'################ cvx had an issue with the optimization. Got prob.status: {prob.status} with u.value {u.value}')
        # PID controller: 
        Ki = 0.0
        Kd = 0.005
        Kp = 0.0002
        integral = 0
        prev_error = 0
        
        error = x0[8]-xss[8]
        
        # Integral, Derivative, and Proportional terms of PID
        integral += error * 1
        derivative = (error - prev_error) / 1
        prev_error = error
        
        # Calculate PID control output
        control_output = Ki * integral + Kd * derivative + Kp * error + uss
        
        # Ensure control output is within valid limits (e.g., non-negative)
        control_output = min(max(0.0000001, control_output), 0.04)
        u_fut = control_output
        print(f'################ We resort to  our PID controller with u_fut = {u_fut}')

    print(f"new result at time index {time_index}: result {u_fut}#########################")
    dss.basal_handler_params['x0_old'] = x0
    
    return u_fut, dss

############################################ Simulate #################################################

# Define DeepC controller parameters/Hyperparameters
basal_handler_params_deepc = {
    'r' : 100,  # input cost weight in the DeePC optimization
    'T_fut' : 10,  # number of timesteps into the future that the DeePC optimization plans for. Also, L = T_past + T_fut
    'T_past' : 9,  # number of timesteps into the past that the DeePC optimization plans for. ######## TODO: Changing this from 10 to 20 broke my code!!! ##############
    'T' : T,       # total number of timesteps used to build the Hankel matrix
    'n' : np.shape(A)[0],  # num of states: 9
    'm' : np.shape(B)[1],  # num of inputs: 2
    'p' : np.shape(C)[0],  # num of outputs: 2
    'which_tau' : 0,
    'input_past' : [],
    'output_past' : [],
    'discretization' : 10,
    'x0_old' : 0
}

# Get training data
df = pd.read_pickle("DeePC_data.pkl")

# Running the simulation with MPC controller
glucose, i, insulin_bolus, m, time = simulate(
    basal_handler=deepc_controller,
    basal_handler_params=basal_handler_params_deepc,
    data_given=data_given_deepc,
    meal_input_modulation_factor=1
)

# Process and analyze results
data = glucose_time_vectors_to_dataframe(glucose=glucose, t=time)

agata = Agata()
results = agata.analyze_glucose_profile(data)
print(results)

TimeInTarget = time_in_target(data)
print('TimeInTarget = ' + str(TimeInTarget) + '%')

plot_results(glucose=glucose, insulin_bolus=insulin_bolus, i=i, m=m, time=time)